[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/012-Seabin.ipynb)

# Seabeam

## Setting up

Lets begin by loading our environment

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

!python3 -m pip install "giee @ git+https://github.com/GP211/2023-fall-class-notebooks.git@d00084220a6501d5fe744869f82e64b3dab9c03b" 

Don't forget to restart your runtime before proceeding

## Looking at the data

The following shows an image of deep seawater bottom
in the Pacific of a sea-floor spreading center produced acoustically
by a device called SeaBeam. Note how we have only acquired data over
a limited range making this an ideal missing data problem.

In [ ]:
from sep_python import default_io, Hypercube, FloatVector
import numpy as np
from sep_plot import Grey
import holoviews as hv
hv.extension('bokeh','matplotlib')
!wget https://raw.githubusercontent.com/rgclapp007/gp211-class-notebooks/main/data/seabin.HH

vec=default_io.vector_from_storage("./seabin.HH")
Grey(vec)

## Estimation with a second derivative

As a first attempt we are going to use the second derivative to fill in the missing data. We are going to set this problem up by first binning the data to a regular gird. Our data fitting goal will then be to make sure the model matches the data at the known locations through the operator $\bf J$. Our model styling goal will assume the model has symetric covariance. We will therefore apply a Laplaian ($\bf L$)
$O(\bf m) =||\bf d - \bf J \bf m||^2 + \epsilon^2 || \bf L \bf m||^2$

In [ ]:
from giee import BoxFilter, ConvOpAdjFilter, convOpAdjData

In [ ]:
import numba
from generic_solver._pyOperator import Operator

Lets begin by writing a function that determines where we have data samples.  We will return two numpy arrays that we will turn into sep vectors, one containing a map containing where we have samples set to 1, other set to 1 where we don't have valies.

In [ ]:
from sep_python import get_sep_vector
from numba import njit
hv.extension('bokeh','matplotlib')

@njit()
def known_data(vec, unknown_value=0):
    known=np.copy(vec)
    unknown=np.copy(vec)
    for i2 in range(vec.shape[0]):
        for i1 in range(vec.shape[1]):
            if vec[i2,i1]==unknown_value:
                known[i2,i1]=0
                unknown[i2,i1]=1
            else:
                known[i2,i1]=1
                unknown[i2,i1]=0
    return known, unknown

k,u=known_data(vec.get_nd_array())
known=get_sep_vector(k,hyper=vec.get_hyper())
unknown=get_sep_vector(u,hyper=vec.get_hyper())

Grey(known)+Grey(unknown)

Next lets create are operators $\bf J$ and $\bf L$.

In [ ]:
from generic_solver._pyOperator import DiagonalOp
from giee import Lap2D
j_op=DiagonalOp(unknown)
reg=Lap2D(vec,vec)

Lets solve are simple regularized inversion problem and display the result.


In [ ]:
from generic_solver  import ProblemL2Linear,ProblemL2LinearReg, LCGsolver, BasicStopper
hv.extension('bokeh','matplotlib')

data=vec.clone()
model=data.clone()
eps=100

problemStop=BasicStopper(niter=1000)
problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
solve_base.run(problemLap1)

Grey(problemLap1.model)

## Problems with our covariance description

Generally not a very satisfactory result.  The problem
is that a second derivative is not an accurate description of
the model covariance.

We can get an idea of the covariance of our problem by averaging over samples
some vector difference away. As a result we get a plot where the axes are now
vector differences and the amplitude tells us about covariance between the samples.

In [ ]:
from sep_python import get_sep_vector
hv.extension('bokeh','matplotlib')
from numba import njit
@njit()
def covFunc(model,nm1,nm2,cov):
    for i2 in range(model.shape[0]):
        for i1 in range(model.shape[1]):
            for im2 in range(-nm2,nm2):
                if i2+im2 >=0 and i2+im2 < model.shape[0]:
                    for im1 in range(-nm1,nm1):
                        if i1+im1 >=0 and i1+im1 < model.shape[1]:
                            cov[im2+nm2,im1+nm1]+=(model[i2,i1]*model[i2+im2,i1+im1])
cov=get_sep_vector(ns=[81,81],os=[-40,-40],ds=[1,1])
hyper=Hypercube.set_with_ns([81,81],os=[-40,-40],ds=[1,1])
covFunc(vec.getNdArray(),40,40,cov.get_nd_array())
Grey(cov,pclip=100)

Note how we the covariance is not as simple as a second derivative.
We see several different linear-like trends. The most dominant dipping slightly down as
we move left to right in the image.

## Prediction Error Filters

We could try to redesign our regularization operator to emulate that trend but a better solution is to use Prediction Error Filters (PEFs) to capture the inverse spectrum.

In general, with Prediction Error Filters, we are attempting to find the filter $\bf f$ that when convolved with the data $\bf D$ gives a minimum energy solution, which would seem to lead to
$O(\bf f) = || \bf D \bf f||^2$. With our filter, we only want to convolve with samples that occur at or before in time (and space). This is what is called a {\it causal} filter.  

The problem is that the obvious answer to this solution is
for $\bf f$ =\bf 0$.  To address this issue, we are going to fix one of the 
coefficients in the filter to be 1, specifically the one associated with zero
lag in time and space.

A second issue is that the minimization equation above isn't correct.  If the data was actually $\bf 0$, even fixing a coefficient we would still get $\bf 0$.  What we really want is $O(\bf f) = || \bf d + \bf D \bf f||^2$, which in terms of the way we have
been solving problems is equivalent to $O(\bf f) = || - \bf d - \bf D \bf f||^2$.

As far as how to construct the shape of $\bf f$.  Remember that the longer $\bf f$ is in the time axis the wide ranger of dips we can recover and the more complex spectra we can model. The more rows we add to $\bf f$ the more dips we can capture.


For the Seabeam problem, we are further challenged because
we can only estimate spectra on locations where
we are filter fully sits on the data. Whenever any of our
filter coefficients are not on a recorded data sample leading to an inaccurate 
fitting equation.  As a result, we are going to add a weighting function  $\bf W$
where $\bf W$ is 0 at any location where one or more coefficients of the
the filters are on unknown data samples. Our new minimization becomes
 $O(\bf f) = ||\bf W( - \bf d - \bf D \bf f)||^2$.  To fit within are
standard inversion scheme we can define a new data $\bf d_{new} = - \bf W \bf d$ and new operator $\bf D_new = \bf W \bf D$.

Below you can find code that estimates a Prediction Error Filter using this approach.

In [ ]:
from generic_solver._pyOperator import DiagonalOp, ChainOperator
from giee import BoxFilter, ConvOpAdjFilter
from sep_python import Hypercube

def findPef(data,known,sh,zero):
    filt=BoxFilter.PEF(sh,zero)
    op=ConvOpAdjFilter(filt,data,data)
    wt=filt.create_mask(known)
    wtOp=DiagonalOp(wt)
    # 
    #  W(d-Lm) = Wd -WLm
    #
    duse=data.clone()
    data.scale(-1.)
    wtOp.forward(False,data,duse)
    wt_pef=ChainOperator(op,wtOp)

    prob=ProblemL2Linear(filt,duse,wt_pef)
    #-d = D f  
    stop=BasicStopper(niter=1000)
    solve=LCGsolver(stop)
    solve.run(prob)
    return prob.model,prob.res,wt


Note in the above code create_mask which returns the weighting function. The trick applied in that function is to set all of our filter coefficients to 1, then convolve a model of 1s.  Any data values that are equal to the number of filter coefficients is a valid regression equation.

We are going to construct a filter which is 7 in the vertical (fast) axis and 4 in the time horizontal axis. Lets first plot the weighting function to see where we have valid equations.

In [ ]:
hv.extension('bokeh','matplotlib')

pef,res,wt=findPef(vec,known,(4,7),(0,3))
Grey(wt)


Note that the locations where we have valid equations are significantly less than where we have data which makes sense given the filter dimensions.

It can also be useful to look at residuals. Note the IID nature of the residual.

In [ ]:
hv.extension('bokeh','matplotlib')
Grey(res)

In [ ]:
## Estimating with the Prediction Error Filter

Now that we have the PEF we can use it to regularize are problem.  Let's solve the problem substituting convolving with the  PEF for the Laplacian in our standard regularized inversion problem.

In [ ]:
hv.extension('bokeh','matplotlib')
from giee import convOpAdjData

j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=convOpAdjData(model,data,pef)
eps=50

problemPEF=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
solve_base.run(problemPEF)

Grey(problemLap1.model,pclip=99)+Grey(problemPEF.model,pclip=99)

## Incorporating variance
Least squares give us the minimum energy solution. As a result, it will tend towards
zero most of the time (depending on boundary conditions). The solutions also tend to go
quite smoothly as we move away from known data locations. In some cases this is a wanted
results. Often it is not.


## Krigging
 In the field of geostastics, the need for introducing morerealistic texture led to the concept of krigging. 

Krigging involves introducing some randomness to an estimate at a given location, not just was what is the best fit given are limited descriptions of the covariance. The basic 
approach to krigging is to randomly select points in the model space and find its nearest neighbors than based on a predefined level of variance and the covariance description choose from a distribution a value for that point. That now becomes a known point and the
process is continued until every point has been visited.

Assuming you change your random seed you can produce many equi-probable models that have more realistic texture than the non-random approach.  Averaging these will also give you
the minimum energy solution.


## Global least square and variance

To apporach the problem from a global least squares problem is a little different.  The key is to consider are residuals.  Specifically lets look at the residual of our model styling goal. Look at the plot below which is the residuals of our Laplcian and PEF based regularized problem.

In [ ]:
hv.extension('bokeh','matplotlib')

Grey(problemLap1.res.vecs[1],pclip=100)+Grey(problemPEF.res.vecs[1],pclip=100)

In the case of the Laplacian problem the residuals are obviously not independent, we can see the obvious linear, dipping trend. In the case of the PEF they are much more IID.  Assuming that we have captured the covariance correctly, what do the amplitudes of the residual represent at locations where have know data?  

A good assumption is that they cary information about other order of stastics, most likely dominated by first-order statistics and the variance.  THis leads to an interesting idea. What happens if we measure those stastics in some way and then somehow put them every in our residual? 

Specifically lets introduce random numbers to are model styling goal and change are minimization to $\bf O(\bf f) = || \bf d -\bf J \bf m||^2 + \epsilon^2 || \bf r -\bf A \bf m||^2$ where $\bf r$ are random numbers with approximately the same distribution as the residual at locations where we had known data.

Lets first figure out a good range for are random numbers.

In [ ]:
res_reg=problemPEF.res.vecs[1]
res_reg.multiply(wt)
tot=np.sum(np.abs(res_reg.get_nd_array()))
nzero=np.sum(wt.get_nd_array())
avg=tot/nzero/eps
rnd=res_reg.clone()
rnd.rand()
rnd.scale(avg*2)

Lets then resolve are regularized inversion problem adding in those random nub,er.

In [ ]:
hv.extension('bokeh','matplotlib')

j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=convOpAdjData(model,data,pef)
eps=50

problemRand1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg,prior_model=rnd)
solve_base=LCGsolver(problemStop)
solve_base.run(problemRand1)

Grey(problemRand1.model,pclip=99)

We can redo the problem and get a different, realistic answer.

In [ ]:
hv.extension('bokeh','matplotlib')

rnd.rand()
rnd.scale(avg*2.)
problemRand2=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg,prior_model=rnd)
solve_base=LCGsolver(problemStop)
solve_base.run(problemRand2)

Grey(problemRand2.model,pclip=99)

If we attempt the same approach where we have not done a good job capturing the covariance the approach stil is interesting but much less satisfying.


In [ ]:
hv.extension('bokeh','matplotlib')

res_reg=problemLap1.res.vecs[1]
res_reg.multiply(wt)
tot=np.sum(np.abs(res_reg.get_nd_array()))
nzero=np.sum(wt.get_nd_array())
avg=tot/nzero/eps
rnd=res_reg.clone()
rnd.rand()
rnd.scale(avg*2)
hv.extension('bokeh','matplotlib')
reg=Lap2D(vec,vec)
j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
eps=50

problemRand3=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg,prior_model=rnd)
solve_base=LCGsolver(problemStop)
solve_base.run(problemRand3)

Grey(problemRand3.model,pclip=99)